# 一、数据清洗

In [1]:
import pymongo
# MongoDB连接配置
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
convoinsight_db = mongo_client["convoinsight"]
conversations_collection = convoinsight_db["conversations"]

## 1. 清洗会话用户情感

In [5]:
#查看用户会话的sentiments的所有情感类型值和频数
# conversations_collection.find_one()
# 查询所有用户消息
from collections import Counter
user_messages = conversations_collection.aggregate([
    {"$unwind": "$messages"},
    {"$match": {"messages.type": "user"}},
    {"$project": {"sentiment": "$messages.sentiment"}}
])

# 提取情感并统计频数
sentiments = [msg['sentiment'] for msg in user_messages if 'sentiment' in msg]
sentiment_counts = Counter(sentiments)

# 输出结果
print(sentiment_counts)

Counter({'中立': 17716, '负向': 7580, '正向': 2575})


In [4]:
# 定义情感映射
sentiment_mapping = {
    '中立': '中立', 'neutral': '中立', 'Neutral': '中立', 'N/A': '中立', '': '中立',
    '负向': '负向', 'negative': '负向', 'Negative': '负向', 'negaitve': '负向', '負向': '负向',
    '正向': '正向', 'positive': '正向', 'Positive': '正向', '积极': '正向',
    # 其他情感类型可以根据需要进行归类
    'frustrated': '负向', 'concerned': '负向', 'disappointed': '负向',
    'apologetic': '负向', 'confused': '中立', '请求': '中立', '要求': '中立',
    'hopeful': '正向', '幽默': '中立', '疑惑': '中立', '疑问': '中立', 'affirmative': '正向'
}

# 更新数据库中的情感数据
for message in conversations_collection.find({"messages.type": "user"}):
    updated_messages = []
    for msg in message['messages']:
        if msg['type'] == 'user' and 'sentiment' in msg:
            original_sentiment = msg['sentiment']
            # 映射情感
            msg['sentiment'] = sentiment_mapping.get(original_sentiment, '中立')
        updated_messages.append(msg)
    
    # 更新数据库
    conversations_collection.update_one(
        {"_id": message["_id"]},
        {"$set": {"messages": updated_messages}}
    )


In [6]:
conversations_collection.find_one()

{'_id': ObjectId('67ceba7fb6cb42d2b375e45d'),
 'agent': '栋栋',
 'customerInfo': {'userId': '15951750186', 'device': None, 'history': None},
 'interactionAnalysis': {'totalMessages': 6,
  'agentMessages': 4,
  'userMessages': 3},
 'messages': [{'type': 'system', 'content': '有新的咨询进来了。', 'sender': '系统'},
  {'type': 'agent',
   'content': '您好，欢迎光临影石insta360~我是栋栋，请问有什么可以帮到您的吗？',
   'sender': '栋栋'},
  {'type': 'user',
   'content': 'X4误闯洗衣机，',
   'sender': '15951750186',
   'sentiment': '中立'},
  {'type': 'user',
   'content': '昨天按照客服说放进米缸中一天后看看',
   'sender': '15951750186',
   'sentiment': '中立'},
  {'type': 'user',
   'content': '现在无法开机',
   'sender': '15951750186',
   'sentiment': '负向'},
  {'type': 'agent',
   'content': '您好，您可以通过点击以下链接，再点击售后服务申请，发起维修申请。 https://www.insta360.com/cn/support/workorder/repair_nav （在App端打开可能会出现填写卡顿的问题，您可以复制该网址至浏览器打开填写） *产品在保修期内出现非人为原因导致问题的情况，我们将提供免费维修服务。 *如已过保修期，具体的维修费用需要您先寄回产品，我们将根据具体检测结果提供维修报价，您可通过https://www.insta360.com/cn/support/workorder/repair_nav至【我的售后工